In [168]:
import pandas as pd
import numpy as np
import summarytools
import mysql.connector


# Inspect Data

In [169]:
# Connect with mysql database
connection = mysql.connector.connect(
    host='localhost',      # Replace with your database host
    user='root',  # Replace with your username
    password='1234',  # Replace with your password
    database='house'   # Replace with your database name
)

query = "SELECT * FROM house"
df = pd.read_sql(query, connection)
connection.close()


summarytools.dfSummary(df)

C:\Users\Ying\AppData\Local\Temp\ipykernel_2380\1775808620.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,address[object],1. 1102 BL (Amsterdamse Poort e.o2. 2132 HE (Hoofddorp Zuid)3. 1062 AC (Overtoomse Veld)4. 1441 DR (Binnenstad)5. 1013 BD (Houthavens)6. 2011 RZ (Bakenes)7. 3527 DZ (Kanaleneiland-Noord)8. 6222 CS (Limmel)9. 5611 EC (Binnenstad)10. 2288 DC (Plaspoelpolder)11. other,"19 (0.9%)11 (0.5%)11 (0.5%)9 (0.4%)6 (0.3%)5 (0.2%)5 (0.2%)5 (0.2%)4 (0.2%)4 (0.2%)2,122 (96.4%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAA6NJREFUeJzt3DFuG1cYRtF/BEKxmiEIAlpCsgctwov1IriI7EEQNEXkqGGKpIngGHDhOwBzTiWwesUFH0nMp+V6vQ4U7vY+AP8fYiMjNjJiIyM2MmIjIzYyYiMjNjLLzDzMzFyv17edz8KNO5zP588zM8uyfBEcP9Pduq6ndV1PM3O/92G4bT6zkREbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERuZw7ZtL//8/b7rSbh5NghkFv8yi4p3NjLWVWSsq8j46YOM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmPhZMxeCHjnY2MwQsZgxcyfvogIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiNj8ELGY+FkDF7I+MxGxjVKxrqKjHUVGZ/ZyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIWFeR8Vg4GesqMj6zkblbluW3ZVk+7X0Qbt/dzPw6M7/sfRBun2uUjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI3M3cz8PjN/7n0Qbp8NAhnXKBmxkTF4IWPwQsY1SkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkrKvIWFeRcY2SERsZ6yoy1lVkXKNkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGQMXsgYvJBxjZIRGxmDFzIGL2Rco2TERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHGNkhEbGbGRsa4iY11FxjVKRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEbGuoqMdRUZ1ygZsZExeCFj8ELGNUpGbGTERkZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERsbghYzBCxnXKJllZh5m5n7vg3BT3q/X69vHFw/n8/nzuq6nPU7Ebdq27WVZli8fgzus63p6enp6Ox6PX/c6HLfj9fX10+VyOT0/P9/PzL9jm5k5Ho9fHx8f/9jldNyih2+96AsCGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbmcPM34+F7H0QbsP3Wjps2/ZyuVxO8x+PhcCP2rbtZWbeP77usXB+hm8+Fm5dRcYXBDJiIyM2MmIjIzYyYiMjNjJiIyM2Mn8BS2qu7EAmsPMAAAAASUVORK5CYII="">",0(0.0%)
2,agent_name[object],1. Verhuurtbeter.nl2. Hausing3. Gruno Vastgoed B.V.4. HouseHunting Eindhoven5. Rotterdam Apartments6. Amsterdam Housing7. HouseHunting Randstad8. Rotsvast Eindhoven9. Huizenbalie.nl10. Rental Rotterdam11. other,"50 (2.3%)36 (1.6%)33 (1.5%)26 (1.2%)24 (1.1%)23 (1.0%)23 (1.0%)22 (1.0%)20 (0.9%)20 (0.9%)1,924 (87.4%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAABClJREFUeJzt3UFqG2cAhuHfQrj2ZoQQ+Bg+gA6Rw+YQOkDuYYxmEbnZuIsqwcmihaJ51arPswkRRMziZRQN/2ffvb+/Dyisrn0B/H+IjYzYyIiNjNjIiI2M2MiIjYzYyNyNMR7HGPcfXvv2/v5+utL1cMPWu93u0zRN2+8vzPP8end391lwXNp6mqbtfr8/bTabt+Px+HA4HLYvLy/3YwyxcVHrMcbYbDZvT09PX8+vPV7xerhhviCQERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRERuZ9RhjHI/Hh49/whLW8zy/Hg6H7TifY5vn+XWM8e26l8UtskEgc+dHZlH59c7mrsZiflpXWVaxpNX3ddV+vz+do7v/238F/8CPddX575ZVLMZDXTJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjI/DV5gSR8HL8YuLMqxcDIGL2Q+3tnc1VjUj8GLsQtLW03TtH1+fh7GLixtNcYYj4+Pv1/7Qrh9HuqSERsZsZERGxmxkREbGbGRERsZsZERGxmxkREbGbGRWY0xxul0+u3aF8LtW83z/PrlyxdjFxbnWDgZgxcyviCQ8TFKxrqKjHUVGesqMr4gkBEbGbGRERsZsZERGxmxkREbGbGRERsZsZERGxmxkbGuImNdRcaxcDLWVWTc2cgYvJAxeCFj8ELGQ10yYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOBZOxuCFjDsbGYMXMgYvZAxeyHioS0ZsZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZKyryFhXkbFBIGNdRcadjYx1FRnrKjLWVWQ81CUjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyYiNj8ELG4IWMY+FkDF7IuLORMXghY/BCxuCFjIe6ZMRGRmxkxEZGbGTERkZsZMRGRmxkxEZGbGTERkZsZAxeyBi8kHEsnIzBCxlfEMh8/xgdPkJZ2nq3230aYwzLKpa2mqZpa1lFwf/ZyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIrM+rqjEsq1iYY+FkrKvIuLORMXghY/BCxqMPMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kHAsnY/BC5uMvSoNfXfSX5613u92n8wYBfjLP8+slh1DraZq2+/3+tNls3i7xhtyG4/H4cDgcti8vL/djjMvENsYYm83m7enp6esl3pCb8njJN/OcjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjcx6jD+Pk1z7Qvh3WaKJ9TzPr4fDYTsufJyE/77zGOpiQyjHwvkrFz0WbvBCxhcEMmIjIzYyYiMjNjJiIyM2MmIjIzYyfwBxJKgWepp2XQAAAABJRU5ErkJggg=="">",0(0.0%)
3,agent_url[object],1. https://www.pararius.com/real-2. h

# Data Cleasing

In [170]:
df = df.drop(columns = ['address', 'agent_name', 'agent_url', 'house_name', 'description', 'house_url', 'maximum_months', 'minimum_months', 'status'])

In [171]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2201 entries, 0 to 2200
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   available             1859 non-null   object 
 1   balcony               2010 non-null   object 
 2   city                  2201 non-null   object 
 3   construction_type     2040 non-null   object 
 4   deposit               1373 non-null   float64
 5   district              2201 non-null   object 
 6   dwelling_type         2201 non-null   object 
 7   energy_rating         1737 non-null   object 
 8   interior              1832 non-null   object 
 9   living_area_m2        2201 non-null   float64
 10  number_of_bathrooms   1540 non-null   float64
 11  number_of_bedrooms    1793 non-null   float64
 12  number_of_rooms       2201 non-null   int64  
 13  offered_since         2201 non-null   object 
 14  pets_allowed          904 non-null    object 
 15  price                

In [172]:
print(df['balcony'].unique()) # fillna with not present
print(df['construction_type'].unique()) # fillna with mode
print(df['dwelling_type'].unique())
print(df['interior'].unique()) # fillna using relationship
# print(df['property_type'].unique())
print(df['rental_agreement'].unique()) # fillna with not specified
print(df['smoking_allowed'].unique())
print(df['pets_allowed'].unique())

['Not present' 'Present' None]
['Existing building' 'New development' None]
['Flat' 'House' 'Room' 'Studio']
['Upholstered' 'Furnished' None 'Upholstered or furnished' 'Shell']
[None 'Unlimited period' 'Temporary rental']
[None 'No' 'Yes']
[None 'No' 'Yes']


In [173]:
df['balcony'].fillna('Not present', inplace = True)
df['construction_type'].fillna(df['construction_type'].mode()[0], inplace = True)
df['rental_agreement'].fillna('Not specified', inplace = True)
df['pets_allowed'].fillna('No', inplace = True)
df['smoking_allowed'].fillna('No', inplace = True)

C:\Users\Ying\AppData\Local\Temp\ipykernel_2380\2735411249.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['balcony'].fillna('Not present', inplace = True)
C:\Users\Ying\AppData\Local\Temp\ipykernel_2380\2735411249.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

In [174]:
# Impute number_of_bedrooms and number_of_bathrooms
median_bathrooms = int(df['number_of_bathrooms'].median())

# Imputation logic
def ImputeBedrooms(row):
    if pd.isnull(row['number_of_bedrooms']):
        # Set to 1 if dwelling type is 'room'
        if row['dwelling_type'] == 'room':
            return 1  
        elif not pd.isnull(row['number_of_rooms']) and not pd.isnull(row['number_of_bathrooms']):
            # If both rooms and bathrooms are not null 
            if row['number_of_rooms'] - row['number_of_bathrooms'] <= 0:
                return row['number_of_rooms'] 
            else:
                return row['number_of_rooms'] - row['number_of_bathrooms']
        elif not pd.isnull(row['number_of_rooms']):
            return row['number_of_rooms']
    return row['number_of_bedrooms']

def ImputeBathrooms(row):
    if pd.isnull(row['number_of_bathrooms']):
        return median_bathrooms 
    return row['number_of_bathrooms']

def TransformRooms(row):
    rooms = row['number_of_bathrooms'] + row['number_of_bedrooms']
    return rooms

# Apply the logic
df['number_of_bedrooms'] = df.apply(ImputeBedrooms, axis=1)
df['number_of_bathrooms'] = df.apply(ImputeBathrooms, axis=1)
df['number_of_rooms'] = df.apply(TransformRooms, axis = 1)

df['number_of_bathrooms'] = df['number_of_bathrooms'].astype('int32')
df['number_of_bedrooms'] = df['number_of_bedrooms'].astype('int32')
df['number_of_rooms'] = df['number_of_rooms'].astype('int32')

In [175]:
# service cost category column
def CategorizeServiceCost(value):
    if pd.isnull(value) or value.lower() == 'none':  # Handle NaN or 'None'
        return 'Not specified'
    value = value.lower()
    if 'electricity' in value and 'gas' in value and 'water' in value and 'internet' in value:
        return 'All included'
    elif 'electricity' in value or 'gas' in value or 'water' in value or 'internet' in value:
        return 'Partial included'
    elif 'excludes' in value:
        return 'Not included'
    else:
        return 'Not specified'

df['service_cost_category'] = df['service_cost'].astype(str).apply(CategorizeServiceCost)

In [176]:
from datetime import datetime, timedelta
def StringToDate(value):
    if not pd.isnull(value):
        date = datetime.strptime(value,'%d-%m-%Y')
        return date

df['available'] = df['available'].apply(StringToDate)
df['offered_since'] = df['offered_since'].apply(StringToDate)

df['offer_to_available'] = (df['available'] - df['offered_since']).dt.days

median_duration = int(df['offer_to_available'].median())
print(median_duration)

def AvailableFillNa(row):
    if pd.isnull(row['available']):
        date = row['offered_since'] + timedelta(days = median_duration)
        return date
    return row['available']

df['available'] = df.apply(AvailableFillNa, axis=1)

df['house_age'] = datetime.now().year - df['year_of_construction']
df['house_age'].fillna(df['house_age'].median(), inplace = True)
df.drop(columns = ['year_of_construction'])


21


C:\Users\Ying\AppData\Local\Temp\ipykernel_2380\904631661.py:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['house_age'].fillna(df['house_age'].median(), inplace = True)


,available,balcony,city,construction_type,deposit,district,dwelling_type,energy_rating,interior,living_area_m2,...,offered_since,pets_allowed,price,property_type,rental_agreement,service_cost,smoking_allowed,service_cost_category,offer_to_available,house_age
0,2024-12-27,Not present,Aalsmeer,Existing building,3500.0,Centrum,Flat,B,Upholstered,81.0,...,2024-12-06,No,1750.0,"Upstairs apartment, Apartment",Not specified,Includes: Service costs,No,Not specified,NaN,19.0
1,2024-12-12,Present,Akersloot,Existing building,3600.0,Akersloot Oost,Flat,A,Furnished,67.0,...,2024-12-10,No,1800.0,"Upstairs apartment, Apartment",Unlimited period,Excludes: Service costs,No,Not included,2.0,14.0
2,2024-12-28,Not present,Alkmaar,Existing building,NaN,Burgemeesterskwartier,Flat,None,None,54.0,...,2024-12-07,No,1100.0,Downstairs apartment,Not specified,None,No,Not specified,NaN,6.0
3,2024-12-19,Not present,Alkmaar,Existing building,NaN,Binnenstad-West,Flat,E,Upholstered,42.0,...,2024-11-28,No,845.0,Downstairs apartment,Not specified,Excludes: Service costs,No,Not included,NaN,45.0
4,2024-12-12,Not present,Alkmaar,Existing building,NaN,Emmakwartier,Flat,A,Furnished,40.0,...,2024-11-28,No,1200.0,"Downstairs apartment, Apartment",Not specified,"Includes: Electricity, Gas, Water",No,Partial included,14.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196,2025-01-31,Not present,Veenendaal,Existing building,1820.0,De Pol,Studio,None,Upholstered,21.0,...,2024-12-05,No,910.0,Studio,Unlimited period,Excludes: Service costs,No,Not included,57.0,86.0
2197,2024-12-12,Present,Zandvoort,Existing building,NaN,Boulevard Midden,Studio,A,Furnished,40.0,...,2024-12-06,No,1300.0,"Studio, Service flat",Temporary rental,"Includes: Electricity, Gas, Internet, Water",No,All included,6.0,2.0
2198,2025-02-01,Not present,Zeist,Existing building,2000.0,Centrumschil-Noord,Studio,D,Furnished,25.0,...,2024-12-13,No,1044.0,"Studio, Apartment",Unlimited period,Excludes: Service costs,No,Not included,50.0,91.0
2199,2025-01-01,Not present,Zoetermeer,Existing building,NaN,Stadscentrum,Studio,A,None,58.0,...,2024-12-02,No,1106.0,Studio,Not specified,None,No,Not specified,30.0,33.0


## Impute energy_rating

In [177]:
def ToIntegerScale(value):
    if value is not None:
        if value == 'G':
            return 1
        elif value == 'F':
            return 2
        elif value == 'E':
            return 3
        elif value == 'D':
            return 4
        elif value == 'C':
            return 5
        elif value == 'B':
            return 6
        elif value == 'A':
            return 7
        elif '+' in value:
            lis = list(value)
            extra = lis.count('+')
            return 7+extra
    return value

df['energy_rating'] = df['energy_rating'].apply(ToIntegerScale)


In [182]:
train_data = df[df['energy_rating'].notna()]
correlation = train_data[['energy_rating', 'house_age', 'living_area_m2']].corr()
print(f"Correlation for house age and living_area_m2: {correlation}")

from scipy.stats import f_oneway

# ANOVA for city
anova_city = f_oneway(*(train_data[train_data['city'] == city]['energy_rating'] for city in train_data['city'].unique()))
print("ANOVA for City:", anova_city)

# ANOVA for dwelling_type
anova_dwelling = f_oneway(*(train_data[train_data['dwelling_type'] == dwelling_type]['energy_rating'] for dwelling_type in train_data['dwelling_type'].unique()))
print("ANOVA for Dwelling Type:", anova_dwelling)

Correlation for house age and living_area_m2:                 energy_rating  house_age  living_area_m2
energy_rating        1.000000  -0.311922       -0.088817
house_age           -0.311922   1.000000       -0.021225
living_area_m2      -0.088817  -0.021225        1.000000
ANOVA for City: F_onewayResult(statistic=np.float64(1.7275085632170124), pvalue=np.float64(1.7852169272406729e-09))
ANOVA for Dwelling Type: F_onewayResult(statistic=np.float64(22.19916064783507), pvalue=np.float64(3.022639099145275e-10))


In [185]:
from sklearn.ensemble import RandomForestClassifier

# Train data (rows without missing energy_rating)
train_data = df[df['energy_rating'].notna()]
X_train = train_data[['dwelling_type', 'city']]
y_train = train_data['energy_rating']

# Test data (rows with missing energy_rating)
test_data = df[df['energy_rating'].isna()]
X_test = test_data[['dwelling_type', 'city']]

# One-hot encoding for categorical features
# One-hot encoding for X_train
X_train_encoded = pd.get_dummies(X_train, drop_first=True)

# One-hot encoding for X_test
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# Align the columns of X_test_encoded with X_train_encoded
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)

# Train model and predict
model = RandomForestClassifier()
model.fit(X_train_encoded, y_train)
df.loc[df['energy_rating'].isna(), 'energy_rating'] = model.predict(X_test_encoded)


In [187]:
df['energy_rating']

0       6.0
1       7.0
2       7.0
3       3.0
4       7.0
       ... 
2196    7.0
2197    7.0
2198    4.0
2199    7.0
2200    4.0
Name: energy_rating, Length: 2201, dtype: float64